# 第1天：AI Agent 基础概念与实践

## 1. 什么是AI Agent？

AI Agent是一个能够感知环境、做出决策并采取行动的智能系统。它具有以下核心特征：

- **自主性(Autonomy)**：能够独立运行，无需人类持续干预
- **反应性(Reactivity)**：能够感知环境变化并做出响应
- **主动性(Proactivity)**：能够主动采取行动实现目标
- **社交能力(Social Ability)**：能够与其他Agent或人类交互

## 2. Agent的五种类型

1. **简单反射型Agent**：基于当前感知直接选择行动
2. **基于模型的反射型Agent**：维护内部状态，理解世界如何运作
3. **基于目标的Agent**：有明确目标，选择能达成目标的行动
4. **基于效用的Agent**：不仅考虑目标，还评估不同状态的效用
5. **学习型Agent**：能从经验中学习，不断改进性能

In [2]:
# 环境设置
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 验证环境
print("OpenAI API Key 已配置:", bool(os.getenv("OPENAI_API_KEY")))

OpenAI API Key 已配置: True


## 3. 创建第一个简单Agent

In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

# 初始化语言模型
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 定义工具函数
def calculator(expression: str) -> str:
    """执行数学计算"""
    try:
        result = eval(expression)
        return f"计算结果: {result}"
    except Exception as e:
        return f"计算错误: {str(e)}"

def get_weather(city: str) -> str:
    """获取天气信息（模拟）"""
    weather_data = {
        "北京": "晴天，温度25°C",
        "上海": "多云，温度22°C",
        "广州": "小雨，温度28°C"
    }
    return weather_data.get(city, "未找到该城市的天气信息")

# 创建工具列表
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="用于执行数学计算，输入数学表达式"
    ),
    Tool(
        name="Weather",
        func=get_weather,
        description="获取城市天气信息，输入城市名称"
    )
]

# 初始化Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

C:\Users\18889\AppData\Local\Temp\ipykernel_38420\3835938454.py:47: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [4]:
# 测试Agent
response = agent.run("北京今天天气怎么样？如果温度超过20度，计算一下25乘以4等于多少？")
print("\n最终回答:", response)

C:\Users\18889\AppData\Local\Temp\ipykernel_38420\2157372142.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("北京今天天气怎么样？如果温度超过20度，计算一下25乘以4等于多少？")




> Entering new AgentExecutor chain...


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


首先获取北京今天的天气信息，然后判断温度是否超过20度，如果超过20度再进行数学计算。
Action: Weather
Action Input: "北京"首先获取北京今天的天气信息，然后判断温度是否超过20度，如果超过20度再进行数学计算。
Action: Weather
Action Input: "北京"
Observation: 晴天，温度25°C
Thought:

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


获取到了北京今天的天气信息，温度为25°C，超过了20度，可以进行数学计算。
Action: Calculator
Action Input: "25*4"获取到了北京今天的天气信息，温度为25°C，超过了20度，可以进行数学计算。
Action: Calculator
Action Input: "25*4"
Observation: 计算结果: 100
Thought:

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


北京今天的天气是晴天，温度为25°C，超过了20度，25乘以4等于100。
Final Answer: 今天北京的天气是晴天，温度为25°C，25乘以4等于100。北京今天的天气是晴天，温度为25°C，超过了20度，25乘以4等于100。
Final Answer: 今天北京的天气是晴天，温度为25°C，25乘以4等于100。

> Finished chain.

最终回答: 今天北京的天气是晴天，温度为25°C，25乘以4等于100。


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## 4. Agent的思考链（Chain of Thought）

Agent使用ReAct（Reasoning + Acting）模式：
1. **Thought（思考）**：分析问题，决定下一步行动
2. **Action（行动）**：选择并执行工具
3. **Observation（观察）**：获取工具执行结果
4. **重复**：继续思考-行动-观察循环，直到得出最终答案

## 5. 今日总结与明日预告

### 今日学习成果
- ✅ 理解了AI Agent的核心概念和特征
- ✅ 掌握了Agent的5种基本类型
- ✅ 使用LangChain创建了第一个能够使用工具的Agent
- ✅ 理解了ReAct思考模式

### 明日学习内容
- 深入学习LangChain框架架构
- 掌握Prompt Templates和Output Parsers
- 实现带有Memory的对话Agent
- 学习Document Loaders和Text Splitters